# Lab 9: Document Analysis

In this assignment, we will learn how to do document classification and clustering



## 1. Example

In this example, we use [20newsgroups](https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset) dataset. Each sample is a document and there are totally 20 classes. 

### 1.1 Load data

In [35]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups

data_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
data_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

print("Train data target labels: {}".format(data_train.target))
print("Train data target names: {}".format(data_train.target_names))

print('#training samples: {}'.format(len(data_train.data)))
print('#testing samples: {}'.format(len(data_test.data)))


Train data target labels: [7 4 4 ... 3 1 8]
Train data target names: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
#training samples: 11314
#testing samples: 7532


### 1.2 Represent documents with TF-IDF represention

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler


#TF-IDF representation for each document
vectorizer = TfidfVectorizer(stop_words='english')
data_train_vectors = vectorizer.fit_transform(data_train.data)
data_test_vectors = vectorizer.transform(data_test.data) 

print(data_train_vectors.shape, data_test_vectors.shape)


(11314, 101322) (7532, 101322)


### 1.3 Use KNN to do document classification

Here, we use the cross-validation method to select $K$.

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score


Xtr = data_train_vectors
Ytr = data_train.target

Xte = data_test_vectors
Yte = data_test.target
print(Yte)

k_range = range(1, 5)
param_grid = dict(n_neighbors=k_range)

clf_knn =  KNeighborsClassifier(n_neighbors=1)

grid = GridSearchCV(clf_knn, param_grid, cv=5, scoring='accuracy')
grid.fit(Xtr, Ytr)

print(grid.best_score_)
print(grid.best_params_)

[ 7  5  0 ...  9  6 15]
0.15290833627341466
{'n_neighbors': 1}


### 1.3 Use Logistic Regression to do document classification
Here, we also use the cross-validation method to select the regularization coefficient. 

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np

#=====training with cross validation======
coeff = range(1, 10)
param_grid = dict(C=coeff)

clf_lr = LogisticRegression(penalty='l2')

grid = GridSearchCV(clf_lr, param_grid, cv=5, scoring='accuracy')
grid.fit(Xtr, Ytr)

print(grid.best_params_)

#=====testing======
clf_lr = LogisticRegression(penalty='l2', C=grid.best_params_['C'])
clf_lr.fit(Xtr, Ytr)

y_pred = clf_lr.predict(Xte)

acc = accuracy_score(Yte, y_pred)
macro_f1 = f1_score(Yte, y_pred, average='macro')
micro_f1 = f1_score(Yte, y_pred, average='micro')

print(acc, macro_f1, micro_f1)

{'C': 9}
0.6909187466808284 0.6814048930533746 0.6909187466808284


## 2. Task: Document Classification and Clustering

In this task, we are going to use [BBCNews](BBC_News_Train.csv) dataset. There are 1490 articles from 5 topics, including tech, business, sport, entertainment, politics. 

* Task 1: Please use KNN and logistic regression to do classification, and compare their performance.

* Task 2: Please use K-means to partition this dataset into 5 clusters and find the representative words in each cluster. 

### 2.1 Load data and represent it with TF-IDF representation

In [39]:
# your code
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
df = pd.read_csv('BBC_News_Train.csv')
y = df['Category']
X = df.drop(['Category'], axis=1)
X_train,X_test,y_train,y_test = train_test_split(X, 
                                                 y,
                                                 test_size=0.2,
                                                 random_state=42)
vectorizer = TfidfVectorizer(stop_words='english')
data_train_vectors = vectorizer.fit_transform(X_train['Text'])
data_test_vectors = vectorizer.transform(X_test['Text']) 

print(data_train_vectors.shape, data_test_vectors.shape)


(1192, 22302) (298, 22302)


### 2.2 Use KNN to do document classification

In [40]:
# your code
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score


k_range = range(1, 5)
param_grid = dict(n_neighbors=k_range)

clf_knn =  KNeighborsClassifier(n_neighbors=1)

grid = GridSearchCV(clf_knn, param_grid, cv=5, scoring='accuracy')
grid.fit(data_train_vectors, y_train)

print(grid.best_score_)
print(grid.best_params_)

0.9270173341303047
{'n_neighbors': 4}


### 2.3 Use Logistic Regression to do document classification

In [41]:
# your code
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
coeff = range(1, 10)
param_grid = dict(C=coeff)

clf_lr = LogisticRegression(penalty='l2')

grid = GridSearchCV(clf_lr, param_grid, cv=5, scoring='accuracy')
grid.fit(data_train_vectors, y_train)

print(grid.best_params_)

#=====testing======
clf_lr = LogisticRegression(penalty='l2', C=grid.best_params_['C'])
clf_lr.fit(data_train_vectors, y_train)

y_pred = clf_lr.predict(data_test_vectors)

acc = accuracy_score(y_test, y_pred)
macro_f1 = f1_score(y_test, y_pred, average='macro')
micro_f1 = f1_score(y_test, y_pred, average='micro')

print(acc, macro_f1, micro_f1)


{'C': 5}
0.9697986577181208 0.9708405690370728 0.9697986577181208


### 2.4 Use K-means to do document clustering and find the 10 most representative words in each cluster. 

In [42]:
# your code
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(data_train_vectors)

terms = vectorizer.get_feature_names_out()

print("\nTop 10 words per cluster:\n")
for i, cluster_center in enumerate(kmeans.cluster_centers_):
    print(f"Cluster {i+1} Topic:")
    top_indices = cluster_center.argsort()[::-1][:10]
    top_terms = [terms[ind] for ind in top_indices]
    print(", ".join(top_terms), "\n")



Top 10 words per cluster:

Cluster 1 Topic:
mr, labour, election, blair, party, said, brown, tax, howard, minister 

Cluster 2 Topic:
film, said, music, people, best, mobile, year, tv, new, uk 

Cluster 3 Topic:
england, wales, ireland, rugby, robinson, france, half, nations, coach, game 

Cluster 4 Topic:
game, said, chelsea, win, world, cup, season, players, arsenal, year 

Cluster 5 Topic:
said, mr, government, growth, eu, economy, year, bank, firm, market 

